<a href="https://colab.research.google.com/github/TheClassicTechno/eegmlJH/blob/main/compressNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install eeghdf

!git clone https://github.com/eegml/eeghdf.git 
!cd eeghdf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'eeghdf' already exists and is not an empty directory.


In [9]:
%pip install ipywidgets 
%pip install widgetsnbextension



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eegml/eeghdf.git to /tmp/pip-req-build-2p9fnnc3
  Running command git clone -q https://github.com/eegml/eeghdf.git /tmp/pip-req-build-2p9fnnc3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
[Errno 2] No such file or directory: 'eeghdf'
/content


In [16]:

!pip install git+https://github.com/eegml/eegvis.git

!git clone https://github.com/eegml/eegvis.git
!cd eegvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eegml/eegvis.git to /tmp/pip-req-build-kg79gkgo
  Running command git clone -q https://github.com/eegml/eegvis.git /tmp/pip-req-build-kg79gkgo
Cloning into 'eegvis'...
remote: Enumerating objects: 669, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 669 (delta 52), reused 52 (delta 28), pack-reused 584
Receiving objects: 100% (669/669), 5.77 MiB | 1.44 MiB/s, done.
Resolving deltas: 100% (437/437), done.


In [5]:
# from bz2 import compress
# from curses import window
# from random import sample
# from subprocess import CompletedProcess
import eeghdf
import gzip
import numpy as np

In [6]:
import matplotlib
import matplotlib.pyplot as plt


In [7]:
import eegvis.stacklineplot as stacklineplot

In [20]:
# configuration
matplotlib.rcParams["figure.figsize"] = (12.0, 8.0)
# for new installs you will need to change DATADIR to point to appropriate folder
DATADIR = "/content/eeghdf/data"

EEGFILE = DATADIR + "/absence_epilepsy.eeg.h5"
#eeg = eeghdf.Eeghdf(EEGFILE)

eeg = eeghdf.Eeghdf("absence_epilepsy.eeg.h5")

In [22]:
sample_rate = eeg.sample_frequency
N = int(sample_rate * 10)  # get 10 seconds of data
sec_offset = 0
offset = int(sample_rate * sec_offset)
print(N, offset)

2000 0


In [23]:

first10 = eeg.rawsignals[0, offset : offset + N]  # returns a numpy array

In [24]:
# want to get the buffer reference to this array (???)

data = first10.data  # or first10.get_buffer(...) if need a slice
databytes = bytes(data)
res = gzip.compress(databytes)
res.hex(' ')

TypeError: ignored

In [29]:

c = len(res)
ratio = len(databytes) / c
ratio

3.02571860816944

In [30]:
# now try shifting bits to quant


def quant_to_bytes(n, arr):
    arr = np.right_shift(arr, n)
    data = arr.data
    return bytes(data)


def compress_ndarray(arr) -> bytes:
    databytes = bytes(arr.data)  # arr.data is memory_view of arr
    return gzip.compress(databytes)

# todo, write an uncompress, to validate that this is loss-less compression


def compress_ratio(arr, # expect initially 1D array
                   quant=8):
    """takes arrlike @arr
    quantitizes this to @quant bits"""
    nbytes = arr.dtype.itemsize  # np.uint16(0).nbytes
    nbits = 8 * nbytes
    rshift = nbits - quant
    shifted_arr = np.right_shift(arr, rshift)
    shifted_arr_bytes = bytes(shifted_arr.data)
    comp_arrbytes = gzip.compress(shifted_arr_bytes)

    return len(shifted_arr_bytes) / len(comp_arrbytes)

In [25]:
# test out np.right_shift on 2d arrays
simple2d = np.array([[2000,1028, -200, 10, 65], [30000, -15000,7000,3500,1700] ], dtype=np.int16)

```
0 -> 0000 # base 2
1 -> 0001
2 -> 0010
3 -> 0011
```
this leaves out how to represent negative numbers using two's complement


In [26]:
simple2d

array([[  2000,   1028,   -200,     10,     65],
       [ 30000, -15000,   7000,   3500,   1700]], dtype=int16)

In [27]:
np.right_shift(simple2d,4)

array([[ 125,   64,  -13,    0,    4],
       [1875, -938,  437,  218,  106]], dtype=int16)

In [28]:
bytes(np.right_shift(simple2d,8)).hex(' ') # python 3.8+

TypeError: ignored

In [ ]:
bytes(simple2d.data).hex(' ')

In [ ]:
# I don't think I needed to really change anything between compress_ratio_1d

def compress_ratio_2d(arr2d, # expect 2D array, broadcasting?
                   quant=8):
    """takes arrlike @arr
    quantitizes this to @quant bits"""
    nbytes = arr2d.dtype.itemsize  # np.uint16(0).nbytes
    nbits = 8 * nbytes
    rshift = nbits - quant
    shifted_arr = np.right_shift(arr2d, rshift)
    shifted_arr_bytes = bytes(shifted_arr.data)
    comp_arrbytes = gzip.compress(shifted_arr_bytes)

    return len(shifted_arr_bytes) / len(comp_arrbytes)




In [ ]:
q8 = compress_ratio(first10, quant=8)
q8

In [ ]:
q12 = compress_ratio(first10, quant=12)
q12

In [ ]:
q16 = compress_ratio(first10, quant=16)
q16

In [ ]:
quants = [1, 2, 4, 6, 8, 10, 12, 14, 16]
ratiosbyQ = [compress_ratio(first10, quant=qq) for qq in quants]

In [ ]:

plt.plot(quants, ratiosbyQ)
plt.title("compression ratio vs quantization number")

In [ ]:
ratiosbyQ

In [ ]:
# subdivide this "epoch" of data
f5_0 = first10[0 : int(sample_rate * 5)]
f5_1 = first10[int(sample_rate * 5) : int(sample_rate * 10)]

In [ ]:
seg_compare = [compress_ratio(f5_0, quant=8), compress_ratio(f5_1)]
seg_compare

In [ ]:
# now plot these
fig = plt.Figure()
axs = fig.subplots(2,1)

In [ ]:
axs[0].plot(first10)
axs[0].set_title('rawsignal int16 10s')

In [ ]:
axs[1].bar([0.0, 0.5], seg_compare, 0.5, align="edge")

In [ ]:
axs[1].set_xlabel("segment")

In [ ]:
axs[1].set_ylabel("compression ratio")
axs[1].set_title("compression ratio of first 5 vs second 5 sec (q=8)")

In [ ]:
fig # will show figure in notebook, otherwise will do nothing

Ok, that seems to make sense. compression ration (CR) via gzip compression can tell the difference between signals in some way.
This was a really obvious difference as first 5 seconds is much more predictable than the second half of the signal.
What about some initial comparions for different types of seizures?

- will need to make a scanning function with moving window
- find seizure times
- start with the 3 example files

In [ ]:
# where are the seizures?

an_df = eeg.edf_annotations_df
an_df
an_df[an_df.text.str.contains("SZ|absence|spasm",case= False,)]

In [ ]:
# see that rows 86 and 88 mark a seizure, and rows 91 and 93 do the same
an_df.iloc[86]

In [ ]:
sz_start_sec = an_df.iloc[86].starts_sec
sz_stop_sec = an_df.iloc[88].starts_sec

In [ ]:
surround_sec = 10.0  # sec
epoch = [sz_start_sec - surround_sec, sz_stop_sec + surround_sec]

In [ ]:
epocharr = eeg.rawsignals[
    0:19, int(epoch[0] * sample_rate) : int(epoch[1] * sample_rate)
]
stacklineplot.stackplot(
    epocharr, epoch[1] - epoch[0], start_time=0, yscale=2.0, topdown=True
)

In [ ]:
class ArrScanner:
    """take data in numpy array form or arr-like
    break it up into small windows of size window_size samples
    apply model_callable"""
    def __init__(
        self,
        model_callable,
        data,  # 2D shape (n_chan, n_samples) np array-like: [np.ndarray | Any], #
        window_size: int, # num samples in window
        batch_size: int, 
        model_dir: str = "",
    ):
        """@arrlike
        W - size of window in samples
        @start sample
        @stop sample
        """
        self.model_callable = model_callable
        self.window_size = window_size
        self.batch_size = batch_size
        self.model_dir = model_dir
        # not using data
        
    def evaluate_model(self, data_arr):
        """divide the data array @data_arr
        data_arr.shape is (num_chan, n_sample)
        non-overlapping windows

        """
        start_samp = 0
        end_samp = data_arr.shape[1]
        
        # whole number of windows in data_arr
        n_windows = end_samp//self.window_size
        W = self.window_size
        model = self.model_callable
        out = [ model(data_arr[:,nn*W:(nn+1)*W]) for nn in range(n_windows)]
            
        return np.array(out,dtype=np.float64)

an_df.iloc[86]


epocharr.shape

In [ ]:
ratios = compress_ratio_2d(epocharr, quant=10)

In [ ]:
# sarr.min(), sarr.max()

In [ ]:
test_epoch = epocharr[0:2,:]
window_size_sec = 1
epoch_size_sec = test_epoch.shape[1]/sample_rate

scanner = ArrScanner(compress_ratio_2d, test_epoch, 
                     int(window_size_sec*sample_rate),1)

In [ ]:
res = scanner.evaluate_model(test_epoch)

In [ ]:
res

In [ ]:
fig = plt.Figure()
axs = fig.subplots(2,1)
stacklineplot.stackplot(test_epoch, seconds=epoch_size_sec,ax=axs[0], topdown=True)
axs[1].plot(np.arange(0,len(res))*epoch_size_sec/len(res), res)
axs[1].set_xmargin(0)
axs[1].set_xlabel(f"compression ration (window={window_size_sec} sec)")
fig